In [79]:
from aiida import load_profile
aiida_profile = load_profile()
aiida_profile.name

'default'

In [80]:
## Loading the some require packages and module
from aiida_kkr.workflows import (combine_imps_wc, kkr_flex_wc,
                                 kkr_imp_wc, kkr_imp_sub_wc
                                )
from aiida_kkr.calculations import KkrCalculation, KkrimpCalculation
from aiida.orm import (Group, load_group, load_node, Dict,
                      )
from aiida_kkr.tools import kkrparams
from aiida.engine import submit

In [89]:
# Note that here for doped 3d atoms into the Bi2Te3 host

group_74 = load_group(74)
## first node of group-64 is for kkr_imp_wc process
# is being used for the host_gf or kkr_flex files

imp_wc_with_gf = load_node(16100)
imp1_wc = group_74.nodes[1]
imp2_wc = group_74.nodes[2]

# gf_host_remote = imp_wc_with_gf.inputs.remote_data_gf
# host_gf_wc = imp_wc_with_gf.get_outgoing(node_class=kkr_flex_wc).all()[0].node


impurity1_output_node = imp1_wc.outputs.workflow_info
impurity2_output_node = imp2_wc.outputs.workflow_info
offset_imp2 = {'index':1}

In [82]:
# kkr_code_iffslurm = kkrflex@iffslurm

In [83]:
scf_node = imp1_wc.get_outgoing(node_class=kkr_imp_sub_wc).all()[0].node
## kkr_flex_wc
host_gf = imp1_wc.inputs.remote_data_gf.get_incoming(node_class=kkr_flex_wc).all()[0].node
kkr_calc = host_gf.get_outgoing(node_class=KkrCalculation).all()[0].node
kkr_flex_kkr_param = kkr_calc.inputs.parameters

In [84]:
kkr_flex_kkr_param = kkr_calc.inputs.parameters.get_dict()
kkr_flex_kkr_param['RCLUSTZ'] *= 1
kkr_flex_kkr_param['NSHELD'] = 2500

In [105]:
settings = combine_imps_wc.get_wf_defaults()
builder = combine_imps_wc.get_builder()
settings['jij_run']= True
settings['dos_run']= False
settings['retrieve_kkrflex'] = False
settings['mag_init']= False
settings['threshold_switch_high_accuracy'] = 1e-2
settings['convergence_criterion'] = 1e-7 #convergence_criterion
settings['threshold_aggressive_mixing']= 0.02
settings['strmix']= 0.005
settings['broyden-number'] = 60
settings['nsteps'] = 100
settings['kkr_runmax'] = 7

Version of workflow: 0.3.1


In [106]:
options = scf_node.inputs.options.get_dict()
options['max_wallclock_seconds'] = 36000*1
options

{'withmpi': True,
 'resources': {'num_machines': 1, 'tot_num_mpiprocs': 48},
 'queue_name': '',
 'max_wallclock_seconds': 36000,
 'custom_scheduler_commands': '#SBATCH --account=jara0191\n\nulimit -s unlimited; export OMP_STACKSIZE=2g'}

In [107]:
builder.impurity1_output_node = impurity1_output_node
builder.impurity2_output_node = impurity2_output_node
builder.offset_imp2 = Dict(dict=offset_imp2)

# scf namespace setup
builder.scf.kkrimp = scf_node.inputs.kkrimp
builder.scf.options = Dict(dict=options)
builder.scf.wf_parameters = scf_node.inputs.wf_parameters

# host_gf namespace setup
builder.host_gf.kkr = host_gf.inputs.kkr
builder.host_gf.options = host_gf.inputs.options
builder.host_gf.params_kkr_overwrite = Dict(dict=kkr_flex_kkr_param) #host_gf.inputs.wf_parameters

builder.wf_parameters_overwrite = Dict(dict=settings)
builder.metadata.label = str(imp1_wc.label+ ':' +imp2_wc.label)

In [108]:
combined_imp_submission = submit(builder)


In [99]:
## Here trying to work with host Au and other impurity
## THe calc is taken from the wasmer
imp_Au_group = load_group(85)
wc_list = list(imp_Au_group.nodes)


In [104]:
imp_wc_1 = wc_list[0]
print('first imp wc : ', imp_wc_1.label)
imp_wc_2 = wc_list[1]
print('econd imp wc : ', imp_wc_2.label)

first imp wc :  Ru:Au
econd imp wc :  Ta:Au


In [12]:
scf_node.inputs.wf_parameters.get_dict()

{'hfield': [0.02, 5],
 'nsteps': 300,
 'strmix': 0.02,
 'aggrmix': 0.05,
 'init_pos': None,
 'mag_init': True,
 'mixreduce': 0.5,
 'kkr_runmax': 5,
 'aggressive_mix': 5,
 'broyden-number': 20,
 'accuracy_params': {'NCHEB': None,
  'NPAN_EQ': None,
  'NPAN_LOG': None,
  'RADIUS_LOGPANELS': None},
 'convergence_criterion': 1e-07,
 'threshold_aggressive_mixing': 0.5}

In [48]:
# flex = imp_wc_with_gf.get_outgoing(node_class=kkr_flex_wc).all()[0].node
# flex.get_outgoing(node_class=KkrCalculation).all()[0].node.inputs.parameters.get_dict()


In [40]:
Bi2Te3_conv_kkr = load_node('23b13d9d-6e02-46c7-b3ac-17033b205b19')
Bi2Te3_conv_kkr.pk

245

In [23]:
# Bi2Te3_conv_kkr.inputs.parameters.get_dict()
#

In [10]:
kkr_flex_kkr_param = kkr_calc.inputs.parameters.get_dict()
kkr_flex_kkr_param['RUNOPT'] = [kkr_flex_kkr_param['RUNOPT'][-1]]

In [14]:
kkr_flex_kkr_param.get

20.0

In [18]:
params = kkrparams